<a href="https://colab.research.google.com/github/DhrumilBhatiya/Persist-Ventures/blob/main/Video_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests
!pip install flask pyngrok scikit-learn pandas requests
!pip install Flask requests pandas scikit-learn
!pip install requests Flask

# Fetch Data From URL

In [ ]:
import requests
import pandas as pd
import numpy as np

# Set the base URL for the SocialVerse API
BASE_URL = "https://api.socialverseapp.com"
HEADERS = {"Flic-Token": "flic_6e2d8d25dc29a4ddd382c2383a903cf4a688d1a117f6eb43b35a1e7fadbb84b8"}

def get_data(endpoint, params=None):
    """ Helper function to fetch data from API """
    response = requests.get(endpoint, headers=HEADERS, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Fetch all viewed posts
viewed_posts = get_data(f"{BASE_URL}/posts/view", params={"page": 1, "page_size": 1000})
liked_posts = get_data(f"{BASE_URL}/posts/like", params={"page": 1, "page_size": 1000})
inspired_posts = get_data(f"{BASE_URL}/posts/inspire", params={"page": 1, "page_size": 1000})
rated_posts = get_data(f"{BASE_URL}/posts/rating", params={"page": 1, "page_size": 1000})
all_posts = get_data(f"{BASE_URL}/posts/summary/get", params={"page": 1, "page_size": 1000})
users_data = get_data(f"{BASE_URL}/users/get_all", params={"page": 1, "page_size": 1000})

# Data Preprocessing

In [ ]:
def to_dataframe(data):
    return pd.DataFrame(data) if data else pd.DataFrame()

viewed_df = to_dataframe(viewed_posts['posts'])
liked_df = to_dataframe(liked_posts['posts'])
inspired_df = to_dataframe(inspired_posts['posts'])
rated_df = to_dataframe(rated_posts['posts'])
all_posts_df = to_dataframe(all_posts['posts'])
users_df = to_dataframe(users_data['users'])

# Data Preprocessing: Normalize, Handle Missing Values, and Create Derived Features
def preprocess_data(df):
    df.fillna('', inplace=True)
    df['category'] = df['category'].apply(lambda x: x.lower() if isinstance(x, str) else '')
    return df

viewed_df = preprocess_data(viewed_df)
liked_df = preprocess_data(liked_df)
inspired_df = preprocess_data(inspired_df)
rated_df = preprocess_data(rated_df)
all_posts_df = preprocess_data(all_posts_df)

# Algorithm Development

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

# Content-based filtering: Recommending based on categories and metadata similarity
def content_based_recommendation(user_posts, all_posts_df):
    all_posts_df['category_encoded'] = LabelEncoder().fit_transform(all_posts_df['category'])
    user_posts['category_encoded'] = LabelEncoder().fit_transform(user_posts['category'])

    user_profile = np.mean(user_posts['category_encoded'].values)
    similarity_scores = cosine_similarity([user_profile], all_posts_df['category_encoded'].values.reshape(-1, 1))
    all_posts_df['similarity'] = similarity_scores.flatten()

    recommended_posts = all_posts_df.sort_values(by='similarity', ascending=False).head(10)
    return recommended_posts[['post_id', 'similarity']]

# Collaborative filtering: Using user ratings or interaction data to recommend
def collaborative_filtering(user_id, all_posts_df, interactions_df):
    user_interactions = interactions_df[interactions_df['user_id'] == user_id]

    interaction_matrix = pd.pivot_table(interactions_df, values='rating', index='user_id', columns='post_id')
    similarity_matrix = cosine_similarity(interaction_matrix.fillna(0))

    user_index = interaction_matrix.index.get_loc(user_id)
    similar_users = similarity_matrix[user_index]

    recommended_posts = []
    for i, similarity in enumerate(similar_users):
        if similarity > 0.5:
            recommended_posts.append(interaction_matrix.columns[i])

    return all_posts_df[all_posts_df['post_id'].isin(recommended_posts)]

# Hybrid recommendation (combining content-based and collaborative)
def hybrid_recommendation(user_id, user_posts, all_posts_df, interactions_df):
    content_recommendations = content_based_recommendation(user_posts, all_posts_df)
    collaborative_recommendations = collaborative_filtering(user_id, all_posts_df, interactions_df)

    combined_recommendations = pd.merge(content_recommendations, collaborative_recommendations, on='post_id', how='outer')
    return combined_recommendations[['post_id', 'similarity']].head(10)


# Evaluation Metrics

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn import linear_model


y_data = all_posts_df['rating_count']
x_data = all_posts_df.iloc[:,6:8]

#Split the all_posts_data for LinearRegression
X_train, X_test,Y_train, Y_test = train_test_split(x_data, y_data,test_size=0.4,random_state=1)
linear_regressor = linear_model.LinearRegression()
linear_regressor.fit(X_train, Y_train)
Y_pred = linear_regressor.predict(X_test)

#Evalute Mean Absolute Error and Root mean square error
def evaluate_recommendations(actual, predicted):
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    return mae, rmse

mae, rmse = evaluate_recommendations(Y_test, Y_pred)
print(f"MAE: {mae}, RMSE: {rmse}")


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/feed', methods=['GET'])
def get_recommended_posts():
    username = request.args.get('username')
    category_id = request.args.get('category_id')
    mood = request.args.get('mood', None)

    # Retrieve user posts based on username and category
    user_posts = all_posts_df[all_posts_df['username'] == username]

    if mood:
        recommendations = hybrid_recommendation(username, user_posts, all_posts_df, rated_df)
    else:
        recommendations = recommendations = all_posts_df[all_posts_df['category'] == category_id].head(10)

    return jsonify(recommendations.to_dict(orient='records'))

if __name__ == '__main__':
    app.run(debug=True,port=8000)
